In [1]:
from modules.data import data_reader, data_loader, time_series_analysis
import pandas as pd
import numpy as np
import os

In [2]:
#evaluation = data_reader.read_data(match=["stock"], location="evaluation")
#prediction = data_reader.read_data(match=["stock"], location="prediction")

In [3]:
def save_dataframe_to_csv(dataframe: pd.DataFrame, file_path: str):
    """
    Saves a pandas DataFrame to a CSV file at the specified location.

    Parameters:
    dataframe (pd.DataFrame): The DataFrame to save.
    file_path (str): The location where the CSV file will be saved.

    Returns:
    None
    """
    try:
        dataframe.to_csv(file_path, index=True)
        print(f"DataFrame saved successfully to {file_path}")
    except Exception as e:
        print(f"An error occurred while saving the DataFrame: {e}")

# Data aggregation

In [4]:
# path to evaluation directory
project_folder = os.getcwd()
directory = os.path.abspath(os.path.join(project_folder, "results", "evaluation_with_mes"))
aggregated_results_directory = os.path.abspath(os.path.join(project_folder, "results", "aggregated_results"))

m = ["r2", "mse", "mae", "rmse", "mda", "mape", "mes"]

In [5]:
# Initialize an empty DataFrame to store the aggregated results
aggregated_evaluation = pd.DataFrame()
aggregated_ranking = pd.DataFrame()

In [6]:
# List all files in the directory
files = [f for f in os.listdir(directory) if f.endswith('.csv')]

In [7]:
# Process each file and add to aggregated evaluation

higher_better = ['r2', 'mda', "mes"]
lower_better = ['mse', 'mae', 'rmse', 'mape']

for file in files:
    # Extract parameters from the file name
    base_name = os.path.splitext(file)[0]
    parameters = base_name.split('_')
    param_dict = {}
    for param in parameters:
        if "=" not in param:
            param_dict["FTS"] = param
        else:
            key, value = param.split('=')
            param_dict[key] = value

    # Read the CSV file into a DataFrame
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    df.rename(columns={"Unnamed: 0" : "model"}, inplace=True)

    
    

    # Add parameter columns to the DataFrame
    for key, value in param_dict.items():
        df[key] = value

    # creating the ranking df
    ranked_df = df.copy()
    ranked_df['model'] = df['model']

    # Rank the models for each metric and add to the new DataFrame
    for metric in higher_better:
        ranked_df[metric] = df[metric].rank(ascending=False, method='min').astype(int)

    for metric in lower_better:
        ranked_df[metric] = df[metric].rank(ascending=True, method='min').astype(int)

    # Append the DataFrame to the aggregated DataFrame
    aggregated_evaluation = pd.concat([aggregated_evaluation, df], ignore_index=True)
    aggregated_ranking = pd.concat([aggregated_ranking, ranked_df], ignore_index=True)


In [8]:
def determine_trend(row):
    if row['linear_trend']:
        return 'linear'
    else:
        return 'no_trend'

data_params = ['SD', 'ED', 'FR', 'T', 'TOD']
unique_data_combinations = aggregated_ranking.drop_duplicates(subset=data_params)[data_params]

new_df = pd.DataFrame(columns = data_params)

for i, data_combination in unique_data_combinations.iterrows():



    type = data_combination["TOD"]
    ticker = data_combination["T"]
    frequency = data_combination["FR"]
    start = data_combination["SD"]
    end = data_combination["ED"]

    if ticker == "USDGBP":
        ticker = "USD/GBP"

    
    data =  data_loader.get_data(type = type, ticker = ticker,
                                    frequency = frequency, start = start,
                                    end = end, rtrn = True)
    

    data_features = time_series_analysis.analyze_time_series(data)
    row = {}
    for k in data_params:
        row[k] = data_combination[k]
    for k in data_features.keys():
        row[k] = data_features[k]

    #return new_df
    new_df = pd.concat([new_df, pd.DataFrame([row])], ignore_index=True)

new_df['trend'] = new_df.apply(determine_trend, axis=1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\data_loader.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='bfill') # imputing the missing values
c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\data_loader.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='bfill') # imputing the missing values
c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\data_loader.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='bfill') # imputing the m

In [9]:
new_df

,SD,ED,FR,T,TOD,stationary,adf_statistic,linear_trend,linear_tau,cyclical_patterns,n_partial_autocorrelations,residual_mean,residual_volatility,trend
0,2020-01-01,2022-01-01,daily,Aggregate,cc,True,-3.406147,True,0.343862,True,13.0,0.0,0.0,linear
1,2022-01-01,2024-01-01,daily,Aggregate,cc,True,-4.468611,True,0.247237,True,7.0,0.0,0.0,linear
2,2020-01-01,2024-01-01,weekly,Aggregate,cc,False,-2.302205,True,0.503660,True,6.0,0.0,0.0,linear
3,2020-01-01,2022-01-01,daily,Delayable,cc,True,-3.973469,True,0.114581,True,13.0,0.0,0.0,linear
4,2022-01-01,2024-01-01,daily,Delayable,cc,True,-3.366926,True,0.154029,True,9.0,0.0,0.0,linear
5,2020-01-01,2024-01-01,weekly,Delayable,cc,True,-4.464921,True,-0.106668,True,4.0,0.0,0.0,linear
6,2018-01-01,2020-01-01,daily,DOWJ,index,True,-22.858303,False,0.001392,False,1.0,0.0,0.0,no_trend
7,2020-01-01,2022-01-01,daily,DOWJ,index,True,-6.669457,False,-0.009499,False,1.0,0.0,0.0,no_trend
8,2022-01-01,2024-01-01,daily,DOWJ,index,True,-21.733232,False,0.049972,True,9.0,0.0,0.0,no_trend
9,1987-01-01,2024-01-01,monthly,DOWJ,index,True,-15.217824,False,-0.009633,False,1.0,0.0,0.0,no_trend


In [10]:
# plotting

"""import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Checking the distribution of the boolean variables
def plot_boolean_distribution(df, bool_columns):
    # Create subplots in a single row, with shared y-axis
    fig, axes = plt.subplots(nrows=1, ncols=len(bool_columns), figsize=(len(bool_columns) * 6, 4), sharey=True)

    # If there's only one boolean column, ensure axes is treated as a list
    if len(bool_columns) == 1:
        axes = [axes]

    for ax, col in zip(axes, bool_columns):
        # Calculate the proportion for each category (True and False)
        prop_data = df[col].value_counts(normalize=True).sort_index()

        # Plot the proportions
        sns.barplot(x=prop_data.index.astype(str), y=prop_data.values, ax=ax)
        ax.set_title(f'Distribution of {col}')
        
        # Set y-axis limit to ensure all plots share the same scale
        ax.set_ylim(0, 1.05)  # Proportions always range from 0 to 1
        
        # Add labels to the bars (displaying proportions)
        for p, proportion in zip(ax.patches, prop_data.values):
            height = p.get_height()
            ax.annotate(f'{proportion:.2f}', (p.get_x() + p.get_width() / 2., height),
                        ha='center', va='center', xytext=(0, 10), textcoords='offset points')

    plt.tight_layout()
    plt.show()

boolean_cols = ['quadratic_trend', 'cyclical_patterns', 'stationary']
plot_boolean_distribution(new_df, boolean_cols)"""


"import matplotlib.pyplot as plt\nimport seaborn as sns\nimport pandas as pd\n\n# 1. Checking the distribution of the boolean variables\ndef plot_boolean_distribution(df, bool_columns):\n    # Create subplots in a single row, with shared y-axis\n    fig, axes = plt.subplots(nrows=1, ncols=len(bool_columns), figsize=(len(bool_columns) * 6, 4), sharey=True)\n\n    # If there's only one boolean column, ensure axes is treated as a list\n    if len(bool_columns) == 1:\n        axes = [axes]\n\n    for ax, col in zip(axes, bool_columns):\n        # Calculate the proportion for each category (True and False)\n        prop_data = df[col].value_counts(normalize=True).sort_index()\n\n        # Plot the proportions\n        sns.barplot(x=prop_data.index.astype(str), y=prop_data.values, ax=ax)\n        ax.set_title(f'Distribution of {col}')\n        \n        # Set y-axis limit to ensure all plots share the same scale\n        ax.set_ylim(0, 1.05)  # Proportions always range from 0 to 1\n        \

In [11]:
# plotting

"""import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Mapping for renaming the bars
rename_map = {
    'cc': 'CHAPS',
    'index': 'Stock index',
    'fx': 'Exchange rate',
    'commodity': 'Commodity'
}

# Modified function to use subplots and meet new requirements
def plot_categorical_distribution(df, cat_columns):
    fig, axes = plt.subplots(1, len(cat_columns), figsize=(12, 4))

    # Loop through categorical columns and respective subplot axes
    for i, col in enumerate(cat_columns):
        # Map the categorical values to renamed bars
        mapped_values = df[col].map(rename_map).fillna(df[col])  # Map values & fill missing ones with original

        # Calculate proportions
        prop_df = mapped_values.value_counts(normalize=True).reset_index()
        prop_df.columns = ['Category', 'Proportion']  # Rename columns for clarity
        sns.barplot(x='Category', y='Proportion', data=prop_df, ax=axes[i])

        # Set titles
        if i == 0:
            axes[i].set_title('Types of data')
        elif i == 1:
            axes[i].set_title('Data frequencies')

        # Remove x-axis label
        axes[i].set_xlabel('')

        # Set y-axis label to 'Proportion'
        axes[i].set_ylabel('Proportion')

        # Adjust y-axis limits to provide more space above bars
        axes[i].set_ylim(0, 1.1 * prop_df['Proportion'].max())

        # Add value labels on top of each bar
        for p in axes[i].patches:
            height = p.get_height()
            axes[i].text(
                p.get_x() + p.get_width() / 2,  # Position at the center of the bar
                height + 0.02,  # Slightly more space above the bar
                f'{height:.2f}',  # Format the proportion value to two decimal places
                ha="center"  # Horizontal alignment
            )

    plt.tight_layout()
    plt.show()

# Example usage
categorical_cols = ['TOD', 'FR']
plot_categorical_distribution(new_df, categorical_cols)"""


'import matplotlib.pyplot as plt\nimport seaborn as sns\nimport pandas as pd\n\n# Mapping for renaming the bars\nrename_map = {\n    \'cc\': \'CHAPS\',\n    \'index\': \'Stock index\',\n    \'fx\': \'Exchange rate\',\n    \'commodity\': \'Commodity\'\n}\n\n# Modified function to use subplots and meet new requirements\ndef plot_categorical_distribution(df, cat_columns):\n    fig, axes = plt.subplots(1, len(cat_columns), figsize=(12, 4))\n\n    # Loop through categorical columns and respective subplot axes\n    for i, col in enumerate(cat_columns):\n        # Map the categorical values to renamed bars\n        mapped_values = df[col].map(rename_map).fillna(df[col])  # Map values & fill missing ones with original\n\n        # Calculate proportions\n        prop_df = mapped_values.value_counts(normalize=True).reset_index()\n        prop_df.columns = [\'Category\', \'Proportion\']  # Rename columns for clarity\n        sns.barplot(x=\'Category\', y=\'Proportion\', data=prop_df, ax=axes[i])\

In [12]:
# plotting

"""import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Function to calculate proportions for each categorical variable
def plot_proportions(data, categorical_col, boolean_col, ax):
    # Cross-tabulate to get counts
    crosstab = pd.crosstab(data[categorical_col], data[boolean_col], normalize='index')
    
    # Plot the normalized counts (proportions)
    crosstab.plot(kind='bar', stacked=True, ax=ax)
    
    # Replace "TOD" and "FR" with custom labels for title and x-axis
    if categorical_col == 'TOD':
        categorical_label = 'Type of data'
    elif categorical_col == 'FR':
        categorical_label = 'Frequency of data'
    else:
        categorical_label = categorical_col
    
    ax.set_title(f'{boolean_col} vs {categorical_label}')
    ax.set_xlabel(categorical_label)
    ax.set_ylabel('Proportion')
    ax.legend(title=boolean_col)
    
    # Get current x-axis labels and replace specific names
    x_labels = [label.get_text() for label in ax.get_xticklabels()]
    new_labels = []
    for label in x_labels:
        if label == 'cc':
            new_labels.append('CHAPS')
        elif label == 'index':
            new_labels.append('Stock index')
        elif label == 'fx':
            new_labels.append('Exchange rate')
        elif label == 'commodity':
            new_labels.append('Commodity')
        else:
            new_labels.append(label)
    
    # Set the new x-axis labels
    ax.set_xticklabels(new_labels)

# Set up the matplotlib figure
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

# Plot for 'quadratic_trend' and 'TOD'
plot_proportions(new_df, 'TOD', 'quadratic_trend', axes[0, 0])

# Plot for 'quadratic_trend' and 'FR'
plot_proportions(new_df, 'FR', 'quadratic_trend', axes[0, 1])

# Plot for 'cyclical_patterns' and 'TOD'
plot_proportions(new_df, 'TOD', 'cyclical_patterns', axes[1, 0])

# Plot for 'cyclical_patterns' and 'FR'
plot_proportions(new_df, 'FR', 'cyclical_patterns', axes[1, 1])

# Plot for 'stationary' and 'TOD'
plot_proportions(new_df, 'TOD', 'stationary', axes[2, 0])

# Plot for 'stationary' and 'FR'
plot_proportions(new_df, 'FR', 'stationary', axes[2, 1])

# Adjust layout
plt.tight_layout()
plt.show()"""


'import matplotlib.pyplot as plt\nimport seaborn as sns\nimport pandas as pd\n\n# Function to calculate proportions for each categorical variable\ndef plot_proportions(data, categorical_col, boolean_col, ax):\n    # Cross-tabulate to get counts\n    crosstab = pd.crosstab(data[categorical_col], data[boolean_col], normalize=\'index\')\n    \n    # Plot the normalized counts (proportions)\n    crosstab.plot(kind=\'bar\', stacked=True, ax=ax)\n    \n    # Replace "TOD" and "FR" with custom labels for title and x-axis\n    if categorical_col == \'TOD\':\n        categorical_label = \'Type of data\'\n    elif categorical_col == \'FR\':\n        categorical_label = \'Frequency of data\'\n    else:\n        categorical_label = categorical_col\n    \n    ax.set_title(f\'{boolean_col} vs {categorical_label}\')\n    ax.set_xlabel(categorical_label)\n    ax.set_ylabel(\'Proportion\')\n    ax.legend(title=boolean_col)\n    \n    # Get current x-axis labels and replace specific names\n    x_labels 

In [13]:
#plotting

"""import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assume you already have the dataframe `new_df`
# new_df = your_dataframe

# 5. Correlation matrix for boolean variables
def plot_boolean_correlation(df, bool_columns):
    corr_matrix = df[bool_columns].corr()
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Correlation matrix of boolean variables')
    plt.show()

plot_boolean_correlation(new_df, boolean_cols)"""

"import pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# Assume you already have the dataframe `new_df`\n# new_df = your_dataframe\n\n# 5. Correlation matrix for boolean variables\ndef plot_boolean_correlation(df, bool_columns):\n    corr_matrix = df[bool_columns].corr()\n    plt.figure(figsize=(8, 6))\n    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)\n    plt.title('Correlation matrix of boolean variables')\n    plt.show()\n\nplot_boolean_correlation(new_df, boolean_cols)"

In [14]:
len(unique_data_combinations)

55

In [15]:
def determine_trend(row):
    if row['linear_trend']:
        return 'linear'
    else:
        return 'no_trend'

def add_data_features(df):
    old_names = list(df.columns)
    data_features = ["stationary", "adf_statistic", "linear_trend",
                                                     "linear_tau",
                                                       "cyclical_patterns", "n_partial_autocorrelations", 'residual_volatility', "residual_mean"]

    data_params = ['SD', 'ED', 'FR', 'T', 'TOD']
    new_df = pd.DataFrame(columns = data_params + data_features)

    

    unique_data_combinations = df.drop_duplicates(subset=data_params)[data_params]


    for i, data_combination in unique_data_combinations.iterrows():



        type = data_combination["TOD"]
        ticker = data_combination["T"]
        frequency = data_combination["FR"]
        start = data_combination["SD"]
        end = data_combination["ED"]

        if ticker == "USDGBP":
            ticker = "USD/GBP"

        
        data =  data_loader.get_data(type = type, ticker = ticker,
                                      frequency = frequency, start = start,
                                        end = end, rtrn = True)
        

        data_features = time_series_analysis.analyze_time_series(data)
        row = {}
        for k in data_params:
            row[k] = data_combination[k]
        for k in data_features.keys():
            row[k] = data_features[k]

        #return new_df
        new_df = pd.concat([new_df, pd.DataFrame([row])], ignore_index=True)
        #return new_df

    output = df.merge(new_df, on = data_params, how = "left")
            
    output['trend'] = output.apply(determine_trend, axis=1)
    output = output.drop(columns=['linear_trend'])
    
    return output




In [16]:
aggregated_ranking_with_data_features = add_data_features(aggregated_ranking)

C:\Users\topco\AppData\Local\Temp\ipykernel_21232\877072396.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame([row])], ignore_index=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\data_loader.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='bfill') # imputing the missing values
c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\data_loader.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='bfill') # imputing the missing values
c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\modules\data\data_loader.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['y'] = df['y'].fillna(method='bfill') # imputing the m

In [17]:
pd.set_option('display.max_columns', 10)
aggregated_ranking_with_data_features

,model,r2,mse,mae,rmse,...,cyclical_patterns,n_partial_autocorrelations,residual_volatility,residual_mean,trend
0,arima,2,2,1,2,...,True,13,0.0,0.0,linear
1,autoregressor,1,1,2,1,...,True,13,0.0,0.0,linear
2,ft_lag_llama,5,5,5,5,...,True,13,0.0,0.0,linear
3,ft_timeGPT,4,4,4,4,...,True,13,0.0,0.0,linear
4,lag_llama,6,6,7,6,...,True,13,0.0,0.0,linear
...,...,...,...,...,...,...,...,...,...,...,...
1332,ft_lag_llama,1,1,1,1,...,False,1,0.0,0.0,no_trend
1333,ft_timeGPT,4,4,4,4,...,False,1,0.0,0.0,no_trend
1334,lag_llama,7,7,7,7,...,False,1,0.0,0.0,no_trend
1335,prophet,3,3,2,3,...,False,1,0.0,0.0,no_trend


In [18]:
# Create the new dataframe excluding the rows based on the specified conditions
exploration_phase_ranking = aggregated_ranking_with_data_features[
    ~(
        (aggregated_ranking_with_data_features["TOD"] == "commodity") &
        (aggregated_ranking_with_data_features["FR"] == "weekly") &
        (aggregated_ranking_with_data_features["FTL"] != 200) &
        (aggregated_ranking_with_data_features["LR"] != 0.0005) &
        (aggregated_ranking_with_data_features["BS"] != 5) &
        (aggregated_ranking_with_data_features["ME"] != 4)
    )
]


In [19]:
"""temp = exploration_phase_ranking[exploration_phase_ranking["TOD"] == "commodity"]
metrics = ["r2", "mse", "mae", "rmse", "mda", "mape", "model"]
group = ["model"]
a = temp[metrics].groupby(group).mean()"""

'temp = exploration_phase_ranking[exploration_phase_ranking["TOD"] == "commodity"]\nmetrics = ["r2", "mse", "mae", "rmse", "mda", "mape", "model"]\ngroup = ["model"]\na = temp[metrics].groupby(group).mean()'

In [20]:
# plotting difference between best model and others


"""df = a
import pandas as pd
import matplotlib.pyplot as plt

# Set the model as the index
df.reset_index(inplace=True)  # This will convert the index into a column
df.set_index('model', inplace=True)

# Initialize a subplot for each column
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))

# List of metrics/columns to iterate through
metrics = ['r2', 'mse', 'mae', 'rmse', 'mda', 'mape']
axes = axes.flatten()

# Plotting the difference for each metric
for i, metric in enumerate(metrics):
    # Get the minimum value (best model) for the current metric
    min_value = df[metric].min()
    
    # Calculate the difference between the best model and all others
    diff = (df[metric] - min_value)/min_value * 100
    
    # Sort by the difference to ensure models are ordered from smallest to largest difference
    diff_sorted = diff.sort_values()
    
    # Plotting the difference as a vertical bar chart
    axes[i].bar(diff_sorted.index, diff_sorted)
    axes[i].set_title(f'Difference from Best for {metric.upper()}')
    axes[i].set_xlabel('Models')
    axes[i].set_ylabel('Difference')
    axes[i].grid(True)  # Adding gridlines
    
    # Set the y-axis range from 0 to 7 for all subplots
    axes[i].set_ylim(0, 700)
    
    # Rotate x-axis labels for readability
    axes[i].tick_params(axis='x', rotation=45)

# Adjust layout
plt.tight_layout()
plt.show()
"""

"df = a\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# Set the model as the index\ndf.reset_index(inplace=True)  # This will convert the index into a column\ndf.set_index('model', inplace=True)\n\n# Initialize a subplot for each column\nfig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))\n\n# List of metrics/columns to iterate through\nmetrics = ['r2', 'mse', 'mae', 'rmse', 'mda', 'mape']\naxes = axes.flatten()\n\n# Plotting the difference for each metric\nfor i, metric in enumerate(metrics):\n    # Get the minimum value (best model) for the current metric\n    min_value = df[metric].min()\n    \n    # Calculate the difference between the best model and all others\n    diff = (df[metric] - min_value)/min_value * 100\n    \n    # Sort by the difference to ensure models are ordered from smallest to largest difference\n    diff_sorted = diff.sort_values()\n    \n    # Plotting the difference as a vertical bar chart\n    axes[i].bar(diff_sorted.index, diff_sorted)\n   

In [21]:
exploration_phase_ranking

,model,r2,mse,mae,rmse,...,cyclical_patterns,n_partial_autocorrelations,residual_volatility,residual_mean,trend
0,arima,2,2,1,2,...,True,13,0.0,0.0,linear
1,autoregressor,1,1,2,1,...,True,13,0.0,0.0,linear
2,ft_lag_llama,5,5,5,5,...,True,13,0.0,0.0,linear
3,ft_timeGPT,4,4,4,4,...,True,13,0.0,0.0,linear
4,lag_llama,6,6,7,6,...,True,13,0.0,0.0,linear
...,...,...,...,...,...,...,...,...,...,...,...
1087,prophet,7,7,7,7,...,False,1,0.0,0.0,no_trend
1088,lag_llama,3,3,1,3,...,False,1,0.0,0.0,no_trend
1089,ft_lag_llama,2,2,2,2,...,False,1,0.0,0.0,no_trend
1090,timeGPT,5,5,5,5,...,False,1,0.0,0.0,no_trend


# Temp for confluence

In [22]:
temp = aggregated_evaluation[np.logical_and(aggregated_evaluation["TOD"] == "fx", aggregated_evaluation["FR"] == "weekly")]
metrics = m + "model"
group = ["model"]
temp[metrics].groupby(group).mean()

TypeError: can only concatenate list (not "str") to list

In [ ]:
# runs linear regression on the aggregated results and prints out coefficients

import statsmodels.api as sm

metric = "mda"
group = ["model", "FR"]

def perform_linear_regression(df, target_col, explanatory_cols):
    # Extract the target variable
    y = df[target_col]
    
    if len(explanatory_cols) == 2:
        X = df[explanatory_cols[0]] + "_" + df[explanatory_cols[1]]
    else:
        X = df[explanatory_cols]

    
    # One-hot encode categorical variables
    X = pd.get_dummies(X, drop_first=True).astype(int)
    
    # Adding a constant term for the intercept
    X = sm.add_constant(X)
    
    # Performing the linear regression
    model = sm.OLS(y, X).fit()
    
    # Printing the regression coefficients
    print("Regression Coefficients:")
    print(model.params)

perform_linear_regression(aggregated_evaluation, metric, group)

Regression Coefficients:
const                    0.376847
arima_monthly           -0.105052
arima_weekly            -0.095904
autoregressor_daily      0.076355
autoregressor_monthly   -0.033258
autoregressor_weekly    -0.001530
ft_lag_llama_daily       0.126026
ft_lag_llama_monthly     0.088110
ft_lag_llama_weekly      0.123697
ft_timeGPT_daily         0.099754
ft_timeGPT_monthly       0.009477
ft_timeGPT_weekly        0.033316
lag_llama_daily          0.098522
lag_llama_monthly        0.134264
lag_llama_weekly         0.116438
prophet_daily            0.127258
prophet_monthly          0.086401
prophet_weekly          -0.006974
timeGPT_daily            0.060345
timeGPT_monthly         -0.017873
timeGPT_weekly          -0.013508
dtype: float64


# Average metrics across all experiments

In [ ]:
metrics = m + "model"
group = ["model"]
model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()
model_ranking

,r2,mse,mae,rmse,mda,mape
model,,,,,,
arima,2.635220,2.635220,2.635220,2.635220,5.075472,2.314465
autoregressor,5.402516,5.402516,5.283019,5.402516,3.949686,5.000000
ft_lag_llama,3.452830,3.452830,3.553459,3.452830,2.886792,3.641509
ft_timeGPT,3.522013,3.522013,3.566038,3.522013,3.522013,3.566038
lag_llama,4.522013,4.522013,4.578616,4.522013,3.213836,4.635220
prophet,4.471698,4.471698,4.509434,4.471698,3.301887,4.880503
timeGPT,3.993711,3.993711,3.874214,3.993711,4.314465,3.962264


In [ ]:
save_dataframe_to_csv(model_ranking, os.path.join(aggregated_results_directory, "model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\model_ranking.csv


# Metrics across type of data and model

In [ ]:
metrics = m +  ["model", "TOD"]
group = ["TOD", "model"]
tod_model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()
tod_model_ranking

r2       mse       mae      rmse       mda  \
TOD       model                                                             
cc        arima          2.955556  2.955556  2.777778  2.955556  2.777778   
          autoregressor  2.111111  2.111111  1.666667  2.111111  2.777778   
          ft_lag_llama   6.044444  6.044444  6.177778  6.044444  4.555556   
          ft_timeGPT     2.866667  2.866667  2.933333  2.866667  3.600000   
          lag_llama      5.355556  5.355556  5.711111  5.355556  4.800000   
          prophet        5.866667  5.866667  5.777778  5.866667  4.133333   
          timeGPT        2.800000  2.800000  2.955556  2.800000  2.977778   
commodity arima          2.500000  2.500000  2.916667  2.500000  5.583333   
          autoregressor  6.666667  6.666667  6.583333  6.666667  4.000000   
          ft_lag_llama   2.666667  2.666667  2.666667  2.666667  2.833333   
          ft_timeGPT     3.833333  3.833333  3.500000  3.833333  4.083333   
          lag_llama      3.583333  3.583333  2.833333  3.583333  3.250000   
          prophet        4.916667  4.916667  4.916667  4.916667  2.416667   
          timeGPT        3.833333  3.833333  4.583333  3.833333  4.333333   
fx        arima          3.166667  3.166667  2.777778  3.166667  6.000000   
          autoregressor  6.500000  6.500000  6.388889  6.500000  4.500000   
          ft_lag_llama   2.777778  2.777778  2.555556  2.777778  2.500000   
          ft_timeGPT     2.833333  2.833333  3.166667  2.833333  2.777778   
          lag_llama      5.111111  5.111111  5.277778  5.111111  2.944444   
          prophet        3.277778  3.277778  3.944444  3.277778  2.944444   
          timeGPT        4.333333  4.333333  3.888889  4.333333  4.888889   
index     arima          2.369048  2.369048  2.488095  2.369048  6.035714   
          autoregressor  6.750000  6.750000  6.797619  6.750000  4.452381   
          ft_lag_llama   2.321429  2.321429  2.488095  2.321429  2.083333   
          ft_timeGPT     3.976190  3.976190  4.000000  3.976190  3.559524   
          lag_llama      4.083333  4.083333  4.071429  4.083333  2.416667   
          prophet        3.916667  3.916667  3.892857  3.916667  3.059524   
          timeGPT        4.583333  4.583333  4.261905  4.583333  4.904762   

                             mape  
TOD       model                    
cc        arima          2.755556  
          autoregressor  1.622222  
          ft_lag_llama   6.266667  
          ft_timeGPT     2.933333  
          lag_llama      5.644444  
          prophet        5.822222  
          timeGPT        2.955556  
commodity arima          3.333333  
          autoregressor  5.583333  
          ft_lag_llama   2.750000  
          ft_timeGPT     3.666667  
          lag_llama      3.416667  
          prophet        5.500000  
          timeGPT        3.750000  
fx        arima          2.111111  
          autoregressor  6.444444  
          ft_lag_llama   2.277778  
          ft_timeGPT     4.111111  
          lag_llama      4.944444  
          prophet        4.222222  
          timeGPT        3.888889  
index     arima          1.976190  
          autoregressor  6.416667  
          ft_lag_llama   2.654762  
          ft_timeGPT     3.773810  
          lag_llama      4.202381  
          prophet        4.428571  
          timeGPT        4.547619

In [ ]:
save_dataframe_to_csv(tod_model_ranking, os.path.join(aggregated_results_directory, "TOD_model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\TOD_model_ranking.csv


# Metrics across Context length and model

In [ ]:
metrics = m + ["model", "CLTS"]
group = ["CLTS", "model"]
clts_model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()

clts_model_ranking

r2       mse       mae      rmse       mda      mape
CLTS model                                                                    
128  arima          2.547170  2.547170  2.547170  2.547170  4.849057  2.188679
     autoregressor  5.490566  5.490566  5.415094  5.490566  4.113208  5.188679
     ft_lag_llama   3.377358  3.377358  3.320755  3.377358  2.735849  3.660377
     ft_timeGPT     3.698113  3.698113  3.773585  3.698113  3.641509  3.698113
     lag_llama      4.339623  4.339623  4.396226  4.339623  2.981132  4.490566
     prophet        4.396226  4.396226  4.547170  4.396226  3.566038  4.773585
     timeGPT        4.150943  4.150943  4.000000  4.150943  4.452830  4.000000
32   arima          2.867925  2.867925  2.924528  2.867925  5.452830  2.566038
     autoregressor  5.339623  5.339623  5.150943  5.339623  3.603774  4.867925
     ft_lag_llama   3.301887  3.301887  3.452830  3.301887  2.962264  3.471698
     ft_timeGPT     3.169811  3.169811  3.226415  3.169811  3.320755  3.377358
     lag_llama      5.075472  5.075472  5.113208  5.075472  3.811321  4.981132
     prophet        4.547170  4.547170  4.528302  4.547170  2.943396  4.811321
     timeGPT        3.698113  3.698113  3.603774  3.698113  4.037736  3.924528
64   arima          2.490566  2.490566  2.433962  2.490566  4.924528  2.188679
     autoregressor  5.377358  5.377358  5.283019  5.377358  4.132075  4.943396
     ft_lag_llama   3.679245  3.679245  3.886792  3.679245  2.962264  3.792453
     ft_timeGPT     3.698113  3.698113  3.698113  3.698113  3.603774  3.622642
     lag_llama      4.150943  4.150943  4.226415  4.150943  2.849057  4.433962
     prophet        4.471698  4.471698  4.452830  4.471698  3.396226  5.056604
     timeGPT        4.132075  4.132075  4.018868  4.132075  4.452830  3.962264

In [ ]:
save_dataframe_to_csv(clts_model_ranking, os.path.join(aggregated_results_directory, "CLTS_model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\CLTS_model_ranking.csv


# Metrics across data frequency and model

In [ ]:
metrics = m + ["model", "FR"]
group = ["FR", "model"]
fr_model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()
fr_model_ranking


r2       mse       mae      rmse       mda  \
FR      model                                                             
daily   arima          2.440476  2.440476  2.392857  2.440476  4.678571   
        autoregressor  5.154762  5.154762  4.976190  5.154762  3.833333   
        ft_lag_llama   4.035714  4.035714  4.130952  4.035714  3.250000   
        ft_timeGPT     3.619048  3.619048  3.416667  3.619048  3.654762   
        lag_llama      4.023810  4.023810  4.250000  4.023810  3.833333   
        prophet        5.154762  5.154762  5.273810  5.154762  2.928571   
        timeGPT        3.571429  3.571429  3.559524  3.571429  4.154762   
monthly arima          2.200000  2.200000  2.400000  2.200000  5.533333   
        autoregressor  6.333333  6.333333  6.333333  6.333333  4.800000   
        ft_lag_llama   1.333333  1.333333  2.000000  1.333333  2.733333   
        ft_timeGPT     3.733333  3.733333  3.533333  3.733333  4.400000   
        lag_llama      3.133333  3.133333  2.466667  3.133333  2.200000   
        prophet        5.933333  5.933333  5.933333  5.933333  2.600000   
        timeGPT        5.333333  5.333333  5.333333  5.333333  4.266667   
weekly  arima          3.016667  3.016667  3.033333  3.016667  5.516667   
        autoregressor  5.516667  5.516667  5.450000  5.516667  3.900000   
        ft_lag_llama   3.166667  3.166667  3.133333  3.166667  2.416667   
        ft_timeGPT     3.333333  3.333333  3.783333  3.333333  3.116667   
        lag_llama      5.566667  5.566667  5.566667  5.566667  2.600000   
        prophet        3.150000  3.150000  3.083333  3.150000  4.000000   
        timeGPT        4.250000  4.250000  3.950000  4.250000  4.550000   

                           mape  
FR      model                    
daily   arima          2.321429  
        autoregressor  4.654762  
        ft_lag_llama   4.416667  
        ft_timeGPT     3.619048  
        lag_llama      3.904762  
        prophet        5.511905  
        timeGPT        3.571429  
monthly arima          2.066667  
        autoregressor  6.266667  
        ft_lag_llama   1.133333  
        ft_timeGPT     3.266667  
        lag_llama      4.733333  
        prophet        5.733333  
        timeGPT        4.800000  
weekly  arima          2.366667  
        autoregressor  5.166667  
        ft_lag_llama   3.183333  
        ft_timeGPT     3.566667  
        lag_llama      5.633333  
        prophet        3.783333  
        timeGPT        4.300000

In [ ]:
save_dataframe_to_csv(fr_model_ranking, os.path.join(aggregated_results_directory, "FR_model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\FR_model_ranking.csv


# Metrics across type of data, frequency and model

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
metrics = m + ["model", "TOD", "FR"]
group = ["FR", "TOD", "model"]
tod_fr_model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()
tod_fr_model_ranking

r2       mse       mae      rmse  \
FR      TOD       model                                                   
daily   cc        arima          2.666667  2.666667  2.433333  2.666667   
                  autoregressor  1.900000  1.900000  1.500000  1.900000   
                  ft_lag_llama   5.566667  5.566667  5.766667  5.566667   
                  ft_timeGPT     3.333333  3.333333  3.266667  3.333333   
                  lag_llama      5.033333  5.033333  5.566667  5.033333   
                  prophet        6.566667  6.566667  6.466667  6.566667   
                  timeGPT        2.933333  2.933333  3.000000  2.933333   
        commodity arima          3.000000  3.000000  3.111111  3.000000   
                  autoregressor  6.777778  6.777778  6.666667  6.777778   
                  ft_lag_llama   2.888889  2.888889  2.777778  2.888889   
                  ft_timeGPT     3.666667  3.666667  3.222222  3.666667   
                  lag_llama      3.666667  3.666667  3.333333  3.666667   
                  prophet        4.666667  4.666667  4.555556  4.666667   
                  timeGPT        3.333333  3.333333  4.333333  3.333333   
        fx        arima          3.222222  3.222222  2.555556  3.222222   
                  autoregressor  7.000000  7.000000  6.777778  7.000000   
                  ft_lag_llama   3.888889  3.888889  3.666667  3.888889   
                  ft_timeGPT     3.555556  3.555556  3.444444  3.555556   
                  lag_llama      3.222222  3.222222  3.555556  3.222222   
                  prophet        3.111111  3.111111  4.666667  3.111111   
                  timeGPT        4.000000  4.000000  3.333333  4.000000   
        index     arima          1.916667  1.916667  2.138889  1.916667   
                  autoregressor  7.000000  7.000000  7.000000  7.000000   
                  ft_lag_llama   3.083333  3.083333  3.222222  3.083333   
                  ft_timeGPT     3.861111  3.861111  3.583333  3.861111   
                  lag_llama      3.472222  3.472222  3.555556  3.472222   
                  prophet        4.611111  4.611111  4.611111  4.611111   
                  timeGPT        4.055556  4.055556  3.888889  4.055556   
monthly commodity arima          1.000000  1.000000  2.333333  1.000000   
                  autoregressor  6.333333  6.333333  6.333333  6.333333   
                  ft_lag_llama   2.000000  2.000000  2.333333  2.000000   
                  ft_timeGPT     4.333333  4.333333  4.333333  4.333333   
                  lag_llama      3.333333  3.333333  1.333333  3.333333   
                  prophet        5.666667  5.666667  6.000000  5.666667   
                  timeGPT        5.333333  5.333333  5.333333  5.333333   
        index     arima          2.500000  2.500000  2.416667  2.500000   
                  autoregressor  6.333333  6.333333  6.333333  6.333333   
                  ft_lag_llama   1.166667  1.166667  1.916667  1.166667   
                  ft_timeGPT     3.583333  3.583333  3.333333  3.583333   
                  lag_llama      3.083333  3.083333  2.750000  3.083333   
                  prophet        6.000000  6.000000  5.916667  6.000000   
                  timeGPT        5.333333  5.333333  5.333333  5.333333   
weekly  cc        arima          3.533333  3.533333  3.466667  3.533333   
                  autoregressor  2.533333  2.533333  2.000000  2.533333   
                  ft_lag_llama   7.000000  7.000000  7.000000  7.000000   
                  ft_timeGPT     1.933333  1.933333  2.266667  1.933333   
                  lag_llama      6.000000  6.000000  6.000000  6.000000   
                  prophet        4.466667  4.466667  4.400000  4.466667   
                  timeGPT        2.533333  2.533333  2.866667  2.533333   
        fx        arima          3.111111  3.111111  3.000000  3.111111   
                  autoregressor  6.000000  6.000000  6.000000  6.000000   
                  ft_lag_llama   1.666667  1.666667  1.444444  1

In [ ]:
save_dataframe_to_csv(tod_fr_model_ranking, os.path.join(aggregated_results_directory, "TOD_FR_model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\TOD_FR_model_ranking.csv


# Metrics across Context length, frequency and model

In [ ]:
metrics = m + ["model", "CLTS", "FR"]
group = ["FR", "CLTS", "model"]
fr_clts_model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()
fr_clts_model_ranking

r2       mse       mae      rmse       mda  \
FR      CLTS model                                                             
daily   128  arima          2.357143  2.357143  2.428571  2.357143  4.678571   
             autoregressor  5.142857  5.142857  5.035714  5.142857  3.857143   
             ft_lag_llama   4.000000  4.000000  3.892857  4.000000  3.142857   
             ft_timeGPT     4.000000  4.000000  3.785714  4.000000  3.678571   
             lag_llama      3.857143  3.857143  4.142857  3.857143  3.285714   
             prophet        4.750000  4.750000  4.928571  4.750000  3.428571   
             timeGPT        3.892857  3.892857  3.785714  3.892857  4.250000   
        32   arima          2.607143  2.607143  2.607143  2.607143  4.821429   
             autoregressor  5.142857  5.142857  4.892857  5.142857  3.500000   
             ft_lag_llama   3.928571  3.928571  3.964286  3.928571  3.357143   
             ft_timeGPT     2.928571  2.928571  2.821429  2.928571  3.357143   
             lag_llama      4.928571  4.928571  5.107143  4.928571  4.964286   
             prophet        5.464286  5.464286  5.535714  5.464286  2.321429   
             timeGPT        3.000000  3.000000  3.071429  3.000000  3.821429   
        64   arima          2.357143  2.357143  2.142857  2.357143  4.535714   
             autoregressor  5.178571  5.178571  5.000000  5.178571  4.142857   
             ft_lag_llama   4.178571  4.178571  4.535714  4.178571  3.250000   
             ft_timeGPT     3.928571  3.928571  3.642857  3.928571  3.928571   
             lag_llama      3.285714  3.285714  3.500000  3.285714  3.250000   
             prophet        5.250000  5.250000  5.357143  5.250000  3.035714   
             timeGPT        3.821429  3.821429  3.821429  3.821429  4.392857   
monthly 128  arima          2.800000  2.800000  3.400000  2.800000  5.800000   
             autoregressor  7.000000  7.000000  7.000000  7.000000  4.400000   
             ft_lag_llama   1.400000  1.400000  1.600000  1.400000  2.400000   
             ft_timeGPT     3.800000  3.800000  3.800000  3.800000  4.200000   
             lag_llama      3.200000  3.200000  2.400000  3.200000  2.800000   
             prophet        3.800000  3.800000  3.800000  3.800000  2.800000   
             timeGPT        6.000000  6.000000  6.000000  6.000000  4.000000   
        32   arima          2.000000  2.000000  1.800000  2.000000  5.800000   
             autoregressor  6.000000  6.000000  6.000000  6.000000  4.600000   
             ft_lag_llama   1.200000  1.200000  2.400000  1.200000  3.600000   
             ft_timeGPT     3.600000  3.600000  3.400000  3.600000  4.600000   
             lag_llama      3.200000  3.200000  2.400000  3.200000  2.200000   
             prophet        7.000000  7.000000  7.000000  7.000000  2.800000   
             timeGPT        5.000000  5.000000  5.000000  5.000000  4.000000   
        64   arima          1.800000  1.800000  2.000000  1.800000  5.000000   
             autoregressor  6.000000  6.000000  6.000000  6.000000  5.400000   
             ft_lag_llama   1.400000  1.400000  2.000000  1.400000  2.200000   
             ft_timeGPT     3.800000  3.800000  3.400000  3.800000  4.400000   
             lag_llama      3.000000  3.000000  2.600000  3.000000  1.600000   
             prophet        7.000000  7.000000  7.000000  7.000000  2.200000   
             timeGPT        5.000000  5.000000  5.000000  5.000000  4.800000   
weekly  128  arima          2.750000  2.750000  2.500000  2.750000  4.850000   
             autoregressor  5.600000  5.600000  5.550000  5.600000  4.400000   
             ft_lag_llama   3.000000  3.000000  2.950000  3.000000  2.250000   
             ft_timeGPT     3.250000  3.250000  3.750000  3.250000  3.450000   
             lag_llama      5.300000  5.300000  5.250000  5.300000  2.600000   
             prophet        4.050000  4.050000  4.200000  4.050000  3.950000   
             timeGPT        4.0500

In [ ]:
metrics = m + ["model", "FR", "TOD"]
group = ["FR", "TOD", "model"]
fr_tod_model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()
fr_tod_model_ranking

r2       mse       mae      rmse  \
FR      TOD       model                                                   
daily   cc        arima          2.666667  2.666667  2.433333  2.666667   
                  autoregressor  1.900000  1.900000  1.500000  1.900000   
                  ft_lag_llama   5.566667  5.566667  5.766667  5.566667   
                  ft_timeGPT     3.333333  3.333333  3.266667  3.333333   
                  lag_llama      5.033333  5.033333  5.566667  5.033333   
                  prophet        6.566667  6.566667  6.466667  6.566667   
                  timeGPT        2.933333  2.933333  3.000000  2.933333   
        commodity arima          3.000000  3.000000  3.111111  3.000000   
                  autoregressor  6.777778  6.777778  6.666667  6.777778   
                  ft_lag_llama   2.888889  2.888889  2.777778  2.888889   
                  ft_timeGPT     3.666667  3.666667  3.222222  3.666667   
                  lag_llama      3.666667  3.666667  3.333333  3.666667   
                  prophet        4.666667  4.666667  4.555556  4.666667   
                  timeGPT        3.333333  3.333333  4.333333  3.333333   
        fx        arima          3.222222  3.222222  2.555556  3.222222   
                  autoregressor  7.000000  7.000000  6.777778  7.000000   
                  ft_lag_llama   3.888889  3.888889  3.666667  3.888889   
                  ft_timeGPT     3.555556  3.555556  3.444444  3.555556   
                  lag_llama      3.222222  3.222222  3.555556  3.222222   
                  prophet        3.111111  3.111111  4.666667  3.111111   
                  timeGPT        4.000000  4.000000  3.333333  4.000000   
        index     arima          1.916667  1.916667  2.138889  1.916667   
                  autoregressor  7.000000  7.000000  7.000000  7.000000   
                  ft_lag_llama   3.083333  3.083333  3.222222  3.083333   
                  ft_timeGPT     3.861111  3.861111  3.583333  3.861111   
                  lag_llama      3.472222  3.472222  3.555556  3.472222   
                  prophet        4.611111  4.611111  4.611111  4.611111   
                  timeGPT        4.055556  4.055556  3.888889  4.055556   
monthly commodity arima          1.000000  1.000000  2.333333  1.000000   
                  autoregressor  6.333333  6.333333  6.333333  6.333333   
                  ft_lag_llama   2.000000  2.000000  2.333333  2.000000   
                  ft_timeGPT     4.333333  4.333333  4.333333  4.333333   
                  lag_llama      3.333333  3.333333  1.333333  3.333333   
                  prophet        5.666667  5.666667  6.000000  5.666667   
                  timeGPT        5.333333  5.333333  5.333333  5.333333   
        index     arima          2.500000  2.500000  2.416667  2.500000   
                  autoregressor  6.333333  6.333333  6.333333  6.333333   
                  ft_lag_llama   1.166667  1.166667  1.916667  1.166667   
                  ft_timeGPT     3.583333  3.583333  3.333333  3.583333   
                  lag_llama      3.083333  3.083333  2.750000  3.083333   
                  prophet        6.000000  6.000000  5.916667  6.000000   
                  timeGPT        5.333333  5.333333  5.333333  5.333333   
weekly  cc        arima          3.533333  3.533333  3.466667  3.533333   
                  autoregressor  2.533333  2.533333  2.000000  2.533333   
                  ft_lag_llama   7.000000  7.000000  7.000000  7.000000   
                  ft_timeGPT     1.933333  1.933333  2.266667  1.933333   
                  lag_llama      6.000000  6.000000  6.000000  6.000000   
                  prophet        4.466667  4.466667  4.400000  4.466667   
                  timeGPT        2.533333  2.533333  2.866667  2.533333   
        fx        arima          3.111111  3.111111  3.000000  3.111111   
                  autoregressor  6.000000  6.000000  6.000000  6.000000   
                  ft_lag_llama   1.666667  1.666667  1.444444  1

In [ ]:
"""metrics = ["r2", "mse", "mae", "rmse", "mda", "mape", "model", "FR", "TOD", "CLTS"]
group = ["FR", "TOD", "CLTS", "model"]
aggregated_ranking[metrics].groupby(group).mean()
fr_tod_model_ranking"""

In [ ]:
save_dataframe_to_csv(fr_tod_model_ranking, os.path.join(aggregated_results_directory, "FR_TOD_model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\FR_TOD_model_ranking.csv


In [ ]:
save_dataframe_to_csv(fr_clts_model_ranking, os.path.join(aggregated_results_directory, "FR_CLTS_model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\FR_CLTS_model_ranking.csv


# breakdown by time-series features

## Breakdown by stationarity

In [ ]:
metrics = m + ["model", "stationary"]
group = ["stationary", "model"]
stationarity_model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()
stationarity_model_ranking

r2       mse       mae      rmse       mda  \
stationary model                                                             
False      arima          3.190476  3.190476  3.000000  3.190476  3.380952   
           autoregressor  2.238095  2.238095  1.761905  2.238095  2.952381   
           ft_lag_llama   6.571429  6.571429  6.571429  6.571429  3.904762   
           ft_timeGPT     2.476190  2.476190  2.714286  2.476190  3.714286   
           lag_llama      5.809524  5.809524  5.857143  5.809524  4.190476   
           prophet        5.285714  5.285714  5.238095  5.285714  4.285714   
           timeGPT        2.428571  2.428571  2.857143  2.428571  3.047619   
True       arima          2.550725  2.550725  2.579710  2.550725  5.333333   
           autoregressor  5.884058  5.884058  5.818841  5.884058  4.101449   
           ft_lag_llama   2.978261  2.978261  3.094203  2.978261  2.731884   
           ft_timeGPT     3.681159  3.681159  3.695652  3.681159  3.492754   
           lag_llama      4.326087  4.326087  4.384058  4.326087  3.065217   
           prophet        4.347826  4.347826  4.398551  4.347826  3.152174   
           timeGPT        4.231884  4.231884  4.028986  4.231884  4.507246   

                              mape  
stationary model                    
False      arima          3.000000  
           autoregressor  1.761905  
           ft_lag_llama   6.571429  
           ft_timeGPT     2.714286  
           lag_llama      5.857143  
           prophet        5.238095  
           timeGPT        2.857143  
True       arima          2.210145  
           autoregressor  5.492754  
           ft_lag_llama   3.195652  
           ft_timeGPT     3.695652  
           lag_llama      4.449275  
           prophet        4.826087  
           timeGPT        4.130435

In [ ]:
save_dataframe_to_csv(stationarity_model_ranking, os.path.join(aggregated_results_directory, "stationarity_model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\stationarity_model_ranking.csv


## Breakdown by trend

In [ ]:
metrics = m + ["model", "trend"]
group = ["trend", "model"]
trend_model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()
trend_model_ranking

r2       mse       mae      rmse       mda  \
trend    model                                                             
linear   arima          2.644444  2.644444  2.511111  2.644444  2.977778   
         autoregressor  2.222222  2.222222  1.888889  2.222222  2.711111   
         ft_lag_llama   6.177778  6.177778  6.200000  6.177778  4.688889   
         ft_timeGPT     2.911111  2.911111  3.044444  2.911111  3.644444   
         lag_llama      5.377778  5.377778  5.555556  5.377778  4.733333   
         prophet        5.844444  5.844444  5.644444  5.844444  3.977778   
         timeGPT        2.822222  2.822222  3.155556  2.822222  2.933333   
no_trend arima          2.631579  2.631579  2.684211  2.631579  5.903509   
         autoregressor  6.657895  6.657895  6.622807  6.657895  4.438596   
         ft_lag_llama   2.377193  2.377193  2.508772  2.377193  2.175439   
         ft_timeGPT     3.763158  3.763158  3.771930  3.763158  3.473684   
         lag_llama      4.184211  4.184211  4.192982  4.184211  2.614035   
         prophet        3.929825  3.929825  4.061404  3.929825  3.035088   
         timeGPT        4.456140  4.456140  4.157895  4.456140  4.859649   

                            mape  
trend    model                    
linear   arima          2.533333  
         autoregressor  1.866667  
         ft_lag_llama   6.088889  
         ft_timeGPT     3.111111  
         lag_llama      5.533333  
         prophet        5.666667  
         timeGPT        3.200000  
no_trend arima          2.228070  
         autoregressor  6.236842  
         ft_lag_llama   2.675439  
         ft_timeGPT     3.745614  
         lag_llama      4.280702  
         prophet        4.570175  
         timeGPT        4.263158

In [ ]:
save_dataframe_to_csv(trend_model_ranking, os.path.join(aggregated_results_directory, "trend_model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\trend_model_ranking.csv


## breakdown by cycle

In [ ]:
metrics = m + ["model", "cyclical_patterns"]
group = ["cyclical_patterns", "model"]
cycle_model_ranking = exploration_phase_ranking[metrics].groupby(group).mean()
cycle_model_ranking

r2       mse       mae      rmse  \
cyclical_patterns model                                                   
False             arima          2.333333  2.333333  2.456790  2.333333   
                  autoregressor  6.654321  6.654321  6.666667  6.654321   
                  ft_lag_llama   2.469136  2.469136  2.493827  2.469136   
                  ft_timeGPT     3.703704  3.703704  3.851852  3.703704   
                  lag_llama      4.382716  4.382716  4.333333  4.382716   
                  prophet        4.012346  4.012346  3.962963  4.012346   
                  timeGPT        4.444444  4.444444  4.234568  4.444444   
True              arima          2.948718  2.948718  2.820513  2.948718   
                  autoregressor  4.102564  4.102564  3.846154  4.102564   
                  ft_lag_llama   4.474359  4.474359  4.653846  4.474359   
                  ft_timeGPT     3.333333  3.333333  3.269231  3.333333   
                  lag_llama      4.666667  4.666667  4.833333  4.666667   
                  prophet        4.948718  4.948718  5.076923  4.948718   
                  timeGPT        3.525641  3.525641  3.500000  3.525641   

                                      mda      mape  
cyclical_patterns model                              
False             arima          5.925926  1.901235  
                  autoregressor  4.185185  6.432099  
                  ft_lag_llama   2.111111  2.395062  
                  ft_timeGPT     3.777778  3.691358  
                  lag_llama      2.617284  4.555556  
                  prophet        3.074074  4.432099  
                  timeGPT        4.691358  4.592593  
True              arima          4.192308  2.743590  
                  autoregressor  3.705128  3.512821  
                  ft_lag_llama   3.692308  4.935897  
                  ft_timeGPT     3.256410  3.435897  
                  lag_llama      3.833333  4.717949  
                  prophet        3.538462  5.346154  
                  timeGPT        3.923077  3.307692

In [ ]:
save_dataframe_to_csv(cycle_model_ranking, os.path.join(aggregated_results_directory, "cycle_model_ranking.csv"))

DataFrame saved successfully to c:\Users\topco\Dokumenti\MSc Banking and Digital Finance UCL\Modules\Dissertation\MSc_dissertation\results\aggregated_results\cycle_model_ranking.csv


# Fine tuning parameter optimization

In [34]:
pd.set_option('display.max_rows', 20)

a = aggregated_evaluation[np.logical_and(np.logical_and(np.logical_and(aggregated_evaluation["FR"] == "weekly", aggregated_evaluation["TOD"] == "commodity"), aggregated_evaluation["ED"] == "2024-01-01"), aggregated_evaluation["model"] == "ft_lag_llama")]

In [35]:
pd.set_option('display.max_rows', 25)

In [36]:
metrics = ["r2", "mse", "mae", "rmse", "mda", "mape", "model", "BS"]
group = ["BS", "model"]
a[metrics].groupby(group).mean()

,,r2,mse,mae,rmse,mda,mape
BS,model,,,,,,
10,ft_lag_llama,-0.011265,0.165222,0.088273,0.262513,0.551724,1.0
20,ft_lag_llama,-0.011265,0.165222,0.088273,0.262513,0.544061,1.0
5,ft_lag_llama,-0.010466,0.135461,0.078055,0.221912,0.507837,1.0


In [37]:
metrics = ["r2", "mse", "mae", "rmse", "mda", "mape", "model", "ME"]
group = ["ME", "model"]
a[metrics].groupby(group).mean()

,,r2,mse,mae,rmse,mda,mape
ME,model,,,,,,
16,ft_lag_llama,-0.011265,0.165222,0.088273,0.262513,0.551724,1.0
4,ft_lag_llama,-0.010466,0.135461,0.078055,0.221912,0.479624,1.0
8,ft_lag_llama,-0.011265,0.165222,0.088273,0.262513,0.578544,1.0


In [38]:
metrics = ["r2", "mse", "mae", "rmse", "mda", "mape", "model", "FTL"]
group = ["FTL", "model"]
a[metrics].groupby(group).mean()

,,r2,mse,mae,rmse,mda,mape
FTL,model,,,,,,
128,ft_lag_llama,-0.001829,0.002205,0.033273,0.046954,0.563218,1.0
200,ft_lag_llama,-0.006867,0.001537,0.032072,0.039211,0.514107,1.0
64,ft_lag_llama,-0.025101,0.491925,0.199473,0.701373,0.524904,1.0


# Prediction visualization

In [39]:
import matplotlib.pyplot as plt

# Assuming `prediction` is your list of dataframes
def plot_dataframes(prediction):
    num_plots = len(prediction)
    fig, axes = plt.subplots(num_plots, 1, figsize=(10, 5 * num_plots))

    # Check if there's only one dataframe to handle the axes object properly
    if num_plots == 1:
        axes = [axes]

    for i, df in enumerate(prediction):
        ax = axes[i]
        df.plot(ax=ax)
        ax.set_title(f'Dataframe {i+1}')
        ax.set_xlabel('Time')
        ax.set_ylabel('Values')

    plt.tight_layout()
    plt.show()

# Example usage with your `prediction` list
plot_dataframes(prediction)

NameError: name 'prediction' is not defined

In [ ]:
prediction2 = prediction[1]
prediction2.reset_index(inplace=True)

In [ ]:
import yfinance as yf
data = yf.download('AAPL', start='2024-07-27 15:29:00', end='2024-07-27 14:40:00', interval='1m')


    
# Filter the data to the specified timeframe
#data_filtered = data.between_time('15:30', '15:39')

# Calculate the minute returns
data['Returns'] = data['Close'].pct_change()

# Drop the first NaN return value
returns = data['Returns'].dropna()

# Convert the returns to a list
returns_list = returns.tolist()

In [ ]:
data

In [ ]:
# Convert the 'timestamp' column to datetime
prediction2['timestamp'] = pd.to_datetime(prediction2['timestamp'])

# Simulate appending 20 data points prior to the first prediction

"""additional_data = {
    "arima": [None] * 10,
    "lag_llama": [None] * 10,
    "autoregressor": [None] * 10,
    "timestamp": pd.date_range(end=prediction2['timestamp'].iloc[0], periods=10, freq='T'),
    "ft_lag_llama": [None] * 10,
    "actual": [0] * 10  # Assuming actual values for these points
}

additional_df = pd.DataFrame(additional_data)

# Concatenate additional data to the original DataFrame
prediction2 = pd.concat([additional_df, prediction2]).reset_index(drop=True)"""

# Set the timestamp column as the index
prediction2.set_index('timestamp', inplace=True)

# Create subplots
fig, axs = plt.subplots(4, 1, figsize=(14, 20), sharex=True)

# Plot actual values and ARIMA predictions
axs[0].plot(prediction2.index, prediction2['actual'], color='red', linewidth=2, label='Actual')
axs[0].plot(prediction2.index, prediction2['arima'], color='blue', linestyle='-', linewidth=1, label='ARIMA')
axs[0].set_title('Actual vs ARIMA Predictions')
axs[0].legend()
axs[0].grid(True, which='both', linestyle='--', linewidth=0.5)

# Plot actual values and Lag Llama predictions
axs[1].plot(prediction2.index, prediction2['actual'], color='red', linewidth=2, label='Actual')
axs[1].plot(prediction2.index, prediction2['lag_llama'], color='green', linestyle='-', linewidth=1, label='Lag Llama')
axs[1].set_title('Actual vs Lag Llama Predictions')
axs[1].legend()
axs[1].grid(True, which='both', linestyle='--', linewidth=0.5)

# Plot actual values and Autoregressor predictions
axs[2].plot(prediction2.index, prediction2['actual'], color='red', linewidth=2, label='Actual')
axs[2].plot(prediction2.index, prediction2['autoregressor'], color='purple', linestyle='-', linewidth=1, label='Autoregressor')
axs[2].set_title('Actual vs Autoregressor Predictions')
axs[2].legend()
axs[2].grid(True, which='both', linestyle='--', linewidth=0.5)

# Plot actual values and FT Lag Llama predictions
axs[3].plot(prediction2.index, prediction2['actual'], color='red', linewidth=2, label='Actual')
axs[3].plot(prediction2.index, prediction2['ft_lag_llama'], color='orange', linestyle='-', linewidth=1, label='FT Lag Llama')
axs[3].set_title('Actual vs FT Lag Llama Predictions')
axs[3].legend()
axs[3].grid(True, which='both', linestyle='--', linewidth=0.5)

# Set the x-axis labels and format
for ax in axs:
    ax.set_xlabel('Timestamp')
    xticks = prediction2.index.to_list()
    ax.set_xticks(xticks)
    ax.set_xticklabels([x.strftime('%H:%M') for x in xticks], rotation=45)

plt.tight_layout()

# Display the plot
plt.show()
